# Pandas

0) Import libraries, check versions

In [137]:
import pandas as pd
import numpy as np

pd.__version__

'1.0.5'

00) Read csv, do primary analysis

In [1]:
df = pd.read_csv("student-mat.csv", sep=";")

df.describe(include='all')

NameError: name 'pd' is not defined

In [139]:
pd.options.display.max_columns = 33

1) How many variants of mother's job are in the dataset? (min 3 variants)

In [140]:
df["Mjob"].unique().size

5

In [141]:
df['Mjob'].value_counts().size

5

In [142]:
df.describe(include='all')["Mjob"][1]

5

2) Find how many girls have good final grade (not less than 15) and have romantic relationship (number and percentage)

In [143]:
# number
df[(df["G3"] >= 15) & (df["romantic"] == "yes") & (df["sex"] == "F")].shape[0]

9

In [144]:
# percentage
df[(df["G3"] >= 15) & (df["romantic"] == "yes") &  (df["sex"] == "F")].shape[0] / df.shape[0] * 100

2.278481012658228

3) Is it better for student's academic performance if his/her parents work at the same job ('other' should be excluded)?

In [145]:
df[(df["Mjob"] == df["Fjob"]) & (df["Fjob"] != "other")]["G3"].median()

12.0

In [146]:
df[(df["Mjob"] != df["Fjob"]) & (df["Fjob"] != "other")]["G3"].median()

10.5

`As we did not clear poor data, it is better to use median metrix instread of mean metrix as it resistants to outliers. `

`Yes, it is better for student's academic performance if his/her parents work at the same job.` 

4) Who is more healthy: students with final grade not less than 15 or not?

In [147]:
df[df["G3"] >= 15]["health"].unique() # just to confirm that data doesnt have outliers.

array([5, 1, 3, 4, 2], dtype=int64)

In [148]:
df[df["G3"] >= 15]["health"].mean()

3.3972602739726026

In [149]:
df[df["G3"] < 15]["health"].mean()

3.590062111801242

`We can see that students with higher graduates have more problems with health than others.`

5) Is there a direct link between going out with friends and final grade?

In [150]:
df.groupby(by=['goout']).agg({'G3': lambda x: x.mean()})

,G3
goout,
1,9.869565
2,11.194175
3,10.961538
4,9.651163
5,9.037736


`Definitely infrequent walks with friends have a positive effect on the final grade of the student`

6) Is the a connection between Mother's education and desire to take higher education?

In [151]:
df.groupby(by=["Medu", "higher"])['higher'].count()

Medu  higher
0     no          1
      yes         2
1     no          6
      yes        53
2     no          7
      yes        96
3     no          5
      yes        94
4     no          1
      yes       130
Name: higher, dtype: int64

`We should make our conclusions on these students, who don't want to get higher education. As we can see, there is not a strong correlation between a mother's education and desire to take higher education. Lets prove it with .corr `

In [170]:
import warnings
warnings.filterwarnings('ignore')

df_temp = df[["Medu", "higher"]]
df_temp['higher'] = df_temp['higher'].astype('category')
df_temp["bool_higher"] = df_temp["higher"].cat.codes
df_temp["Medu"].corr(df_temp['bool_higher']) # return correlation between 2 values

0.16884453374475855

`As stated above, the correlation is not really strong`

7) Could we say that girls have more failures when they have romantic relationship than boys?

In [207]:
df.groupby(by=["sex", "failures"]).agg({'romantic': lambda x: x.value_counts(normalize=True)["yes"]})

romantic
sex failures          
F   0         0.339286
    1         0.521739
    2         0.818182
    3         0.166667
M   0         0.270833
    1         0.296296
    2         0.333333
    3         0.400000

``

8) Find pairs of jobs with the most and the least average of absences.

In [252]:
df.groupby(["Mjob", "Fjob"]).agg(["max", "min"])["absences"].idxmax()

max    (other, services)
min    (health, teacher)
dtype: object

`As we can see, these sort of job (other, services) has max value in absences, while (health, teacher) has min value. However it is not clear to understand others professions, so lest remove it and check what we will get`

In [276]:
df[(df["Mjob"] != "other") & (df["Fjob"] != "other")].groupby(["Mjob", "Fjob"]).agg(["max", "min"])["absences"].idxmax()

max    (teacher, services)
min      (health, teacher)
dtype: object

`So now it is a little  bit clear`

**!!!OOPPSSS!!!**


`Hah, it is a mistace! I have found just max and min absences, but we need most and the least average of absences so...`

In [277]:
df.groupby(["Mjob", "Fjob"]).agg("mean")["absences"].idxmax()

('other', 'services')

In [274]:
df.groupby(["Mjob", "Fjob"]).agg("mean")["absences"].idxmin()

('at_home', 'at_home')

9) Calulate the average final grade by romantic relationship and availability of Internet access for boys and girls. Who has the biggest rate?

In [260]:
df.groupby(["sex", "romantic"]).agg('mean')["G3"]

sex  romantic
F    no          10.550388
     yes          9.012658
M    no          11.111940
     yes         10.415094
Name: G3, dtype: float64

`The highest value has boys without relationships, so it is better to stay single in order to get more knowledges and higher marks ^_^`

10) Print statistics for free time after school by Mother's and Father's job . Find combination with the biggest average rate. (one of the variants that I would like to see - using describe())

In [279]:
df.groupby(["Mjob", "Fjob"]).agg(["mean"])["freetime"].idxmax()

mean    (teacher, at_home)
dtype: object

In [293]:
df.groupby(["Mjob", "Fjob"]).describe()["freetime"]["mean"].idxmax()

# You will get describe if you want it -_-

('teacher', 'at_home')

`So, the biggest average rate can be obtained by these conbination: ('teacher', 'at_home')`

11) Calculate average rate of educatuin for each job for Father for different family sizes.

In [299]:
df.groupby(["Fjob", "famsize"]).agg("mean")["G3"]

Fjob      famsize
at_home   GT3         9.083333
          LE3        11.750000
health    GT3        11.750000
          LE3        11.333333
other     GT3         9.954248
          LE3        10.765625
services  GT3        10.125000
          LE3        10.741935
teacher   GT3        11.541667
          LE3        14.000000
Name: G3, dtype: float64

12) Calculate average final grade by quality of family relationship and parent's cohabitation status. Which case has the biggest average grade?

In [301]:
df.groupby(["Pstatus", "famrel"]).agg("mean")["G3"]

Pstatus  famrel
A        1         11.500000
         2         11.000000
         3          9.666667
         4         10.529412
         5         12.769231
T        1         10.333333
         2          9.666667
         3         10.080645
         4         10.342697
         5         10.559140
Name: G3, dtype: float64

13) What is the smallest age in a dataset? How many students of this age? What is the percentage of students with final grade not less than 15? What is the percentage of boys and girls from this slice?

In [305]:
# What is the smallest age in a dataset?
df["age"].min()

15

In [313]:
# How many students of this age
df[df["age"] == 15].shape[0]

82

In [317]:
# What is the percentage of students with final grade not less than 15?
df[df["G3"] >= 15].shape[0] / df.shape[0] * 100

18.48101265822785

In [331]:
# What is the percentage of boys and girls from this slice?
df["sex"].value_counts(normalize=True)

F    0.526582
M    0.473418
Name: sex, dtype: float64

#### Please, write not only numbers but answers also if it's needed :)